## Title: World Dryland Dataset 2007

### Description
World dryland areas according to UNCCD and CBD definitions. The original drylands dataset was developed to define the Millennium Ecosystem Assessment (MA) dryland system boundary. Sörensen (2007) used this map to further delineate dryland areas of relevance to the CBD Programme of Work on Dry and Subhumid Lands. This current dataset builds on these maps to incorporate additional features to update the data to better reflect dry and sub- humid tropical forests in accordance with paragraph 13 of Convention on Biological Diversity (CBD) CoP Decision IX/17.<br>
The original dataset developed by the MA is a subset of the aridity zone map created by CRU/UEA, UNEP/GRID for the World Atlas of Desertification. Aridity zones were derived from an Aridity Index (AI) which is calculated as the ratio of precipitation to potential evapotranspiration. Zones included in this dataset are: hyperarid, arid, semiarid, and dry subhumid. These are in accordance with the United Nations Convention to Combat Desertification (UNCCD) definition of Drylands.

This dataset was updated in 2014 to fix topological errors.

### FLINT
This dataset has been pre-processed/checked and is suitable for use in FLINT. Please adhere to individual dataset licence conditions and citations. Processed data can be accessed here: https://datasets.mojaglobal.workers.dev/ <br>

### Format
<b>Extent: </b>Global coverage<br>
<b>Format</b>: polygon geoJSON .json<br>
<b>Cordinate system:</b> EPSG:4326 (WGS84)<br>
<b>Year: </b>2007<br>
<b>Size:</b> 20MB 

### Original source
Original Source: https://www.unep-wcmc.org/resources-and-data/world-dryland-areas-according-to-unccd-and-cbd-definitions
submit name, email and organisation for access to the data Accessed 13/12/2020 <br>


### Licence
All uses. UNEP rules: Credit source, Only most recent version can be used, Commercial use only with written permission.
No sub-licensing, No redistribution, No distribution of derivative work.<br>
No publication in downloadable format. See Additional permission info: [Github](https://github.com/moja-global/Land-Sector-Datasets/blob/master/Additional-Permissions/Gmail%20-%20Thank%20you%20for%20useful%20input%20and%20follow%20up.pdf)<br>

Global analyses require notification to the Director of UNEP-WCMC to explore the appropriateness, collaboration, and to avoid duplication of effort. Published materials should be reviewed by UNEP-WCMC prior to publication. Two free copies of all published materials have to be provided to UNEP-WCMC.<br>

1) Adhere to terms and conditions of access <br>
2) acknowledge the source appropriately<br>
3) do not re-distribute part or all of data without contacting UNEP-WCMC

### Citation
UNEP-WCMC, 2007 A spatial analysis approach to the global delineation of dryland areas of relevance to the CBD Programme of Work on Dry and Subhumid Lands. Dataset based on spatial analysis between WWF terrestrial ecoregions (WWF-US, 2004) and aridity zones (CRU/UEA; UNEPGRID, 1991). Dataset checked and refined to remove many gaps, overlaps and slivers (July 2014).

### Original format
Global coverage, vector, shapefile<br>
Cordinate system EPSG: 4326 (WGS84)

### Metadata
https://www.unep-wcmc.org/resources-and-data/world-dryland-areas-according-to-unccd-and-cbd-definitions

This dataset was updated in July 2014 to fix topological errors.

### Notes
No topoloogy errors in original (2014 version). Vector covers dryland areas only. Dataset has valid gaps.

### Processing
Repair geometry, convert to geojson, EPSG:4326 (WGS84), remove/disable Z values. View code below - originally processed in ArcGIS but can be converted to open source QGIS or GDAL (or others).

In [ ]:
import arcpy
import os

#folder of feature classes to process
in_folder = r"C:\Users\LennyJenny\Documents\ArcGIS\world\UNFCCC\downloads\World Dryland Areas\Drylands_dataset_2007\Drylands_latest_July2014"
scr_folder = r"C:\data\scratch.gdb"
out_folder = r"C:\data\json"

# Set environments
workspace = in_folder
arcpy.env.workspace = workspace
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)
arcpy.env.outputZFlag = "Disabled"
arcpy.env.overwriteOutput = True

# Execute CreateFileGDB
if os.path.exists(r"C:\data\scratch.gdb"):
    arcpy.Delete_management(r"C:\data\scratch.gdb")
    scr = arcpy.CreateFileGDB_management(r"C:\data", "scratch")

# Iterate through all datasets, repair and convert to json
featureclasses = arcpy.ListFeatureClasses()
print(featureclasses)

for fc in featureclasses:
    fcname = os.path.join(os.path.splitext(fc)[0])
    outjson = os.path.join(out_folder, fcname)
    
    # Process: Make Feature Layer
    fLayer = "project_Layer"
    arcpy.management.MakeFeatureLayer(fc, fLayer)

    # Process: Intersect - find and fix overlaps
    projectIntersect = os.path.join(scr_folder, "projectIntersect")
    arcpy.analysis.Intersect(fLayer, projectIntersect, "ONLY_FID")
    arcpy.GetCount_management(projectIntersect)
    if arcpy.management.GetCount(projectIntersect)[0] == "0":
        print('No overlaps found')
    else:
        print ('Overlaps found...repairing now...')
    
    # Process: Repair Geometry (non-simple geometry)
    geomRepair = arcpy.management.RepairGeometry(fLayer, "DELETE_NULL", "OGC")[0]

    # Process: Features To JSON
    arcpy.conversion.FeaturesToJSON(fLayer, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")
    print(outjson, '.geojson complete')
    

arcpy.AddMessage("All done!")
print('done')